<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/SachsLab/MonkeyPFCSaccadeStudies/blob/master/StudyLocationRule/Analysis/04_analyze_shallow.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/SachsLab/IMonkeyPFCSaccadeStudies/blob/master/StudyLocationRule/Analysis/04_analyze_shallow.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

# Decode Intended Saccade Direction from Macaque PFC Microelectrode Recordings with Shallow ML

## Normalize Environments
Run the first two cells to normalize Local / Colab environments.

In [ ]:
from pathlib import Path
import os
import sys
import datetime
import numpy as np
import matplotlib.pyplot as plt

try:
    from google.colab import files
    %tensorflow_version 2.x  # Only on colab
    os.chdir('..')
    
    if not (Path.home() / '.kaggle').is_dir():
        # Configure kaggle
        uploaded = files.upload()  # Find the kaggle.json file in your ~/.kaggle directory.
        if 'kaggle.json' in uploaded.keys():
            !mkdir -p ~/.kaggle
            !mv kaggle.json ~/.kaggle/
            !chmod 600 ~/.kaggle/kaggle.json
    
    !pip install git+https://github.com/SachsLab/indl.git
    
    if Path.cwd().stem == 'MonkeyPFCSaccadeStudies':
        os.chdir(Path.cwd().parent)

    if not (Path.cwd() / 'MonkeyPFCSaccadeStudies').is_dir():
        !git clone --single-branch --recursive https://github.com/SachsLab/MonkeyPFCSaccadeStudies.git
        sys.path.append(str(Path.cwd() / 'MonkeyPFCSaccadeStudies'))
    os.chdir('MonkeyPFCSaccadeStudies')
    
    !pip install -q kaggle
    plt.style.use('dark_background')
    IN_COLAB = True
except ModuleNotFoundError:
    import sys
    
    # chdir to MonkeyPFCSaccadeStudies
    if Path.cwd().stem == 'Analysis':
        os.chdir(Path.cwd().parent.parent)
    
    # Add indl repository to path.
    # Eventually this should already be pip installed, but it's still under heavy development so this is easier for now.
    check_dir = Path.cwd()
    while not (check_dir / 'Tools').is_dir():
        check_dir = check_dir / '..'
    indl_path = check_dir / 'Tools' / 'Neurophys' / 'indl'
    sys.path.append(str(indl_path))
    
    # Make sure the kaggle executable is on the PATH
    os.environ['PATH'] = os.environ['PATH'] + ';' + str(Path(sys.executable).parent / 'Scripts')
    
    IN_COLAB = False

# Try to clear any logs from previous runs
if (Path.cwd() / 'logs').is_dir():
    import shutil
    try:
        shutil.rmtree(str(Path.cwd() / 'logs'))
    except PermissionError:
        print("Unable to remove logs directory.")

# Additional imports
import tensorflow as tf
from indl.display import turbo_cmap
plt.rcParams.update({
    'axes.titlesize': 24,
    'axes.labelsize': 20,
    'lines.linewidth': 2,
    'lines.markersize': 5,
    'xtick.labelsize': 16,
    'ytick.labelsize': 16,
    'legend.fontsize': 18
})

%load_ext autoreload
%autoreload 2

In [ ]:
# Download and unzip data
if IN_COLAB:
    data_path = Path.cwd() / 'data' / 'monkey_pfc' / 'converted'
else:
    data_path = Path.cwd() / 'StudyLocationRule' / 'Data' / 'Preprocessed'

if not (data_path).is_dir():
    !kaggle datasets download --unzip --path {str(data_path)} cboulay/macaque-8a-spikes-rates-and-saccades
    print("Finished downloading and extracting data.")
else:
    print("Data directory found. Skipping download.")

## Parameterize

### Data Import

We will set the parameters for our `load_macaque_pfc` function.
Specifically, we are getting the spikerates, which have been smoothed and downsampled, only from trials where the outcome was correct and the d' was at least 1.0.

In [ ]:
from misc.misc import load_macaque_pfc, sess_infos

load_kwargs = {
    'valid_outcomes': (0,),  # Use (0, 9) to include trials with incorrect behaviour
    'zscore': True,
    'dprime_range': (1.0, np.inf),  # Use (-np.inf, np.inf) to include all trials.
    'verbose': True,
    'y_type': 'sacClass',
    'samples_last': True  # Our EEGNet blocks expect time-samples in the last dimension.
}

### Shallow ML Model

We will use a 1-layer logistic regression to classify saccade target from presaccadic activity.

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression

log_reg_kwargs = {
    'solver': 'lbfgs',
    'C': 10.0,  # inverse regularization strength
    'penalty': 'l2',
    'multi_class': 'ovr',
    'max_iter': 1000
}
N_SPLITS = 10

## ML Analysis

### Calculate classification accuracy for each session

In [ ]:
def get_accuracies(sess_infos, time_range=(-np.inf, np.inf),
                  log_reg_kwargs={'solver': 'lbfgs','C': 10.0,'penalty': 'l2',
                                  'multi_class': 'ovr', 'max_iter': 500}):
    
    splitter = StratifiedKFold(n_splits=N_SPLITS, shuffle=True)
    model = LogisticRegression(**log_reg_kwargs)

    accs_out = []
    for sess_ix, sess_info in enumerate(sess_infos):
        sess_id = sess_info['exp_code']
        print(f"\nProcessing session {sess_id}")
        X, Y, ax_info = load_macaque_pfc(data_path, sess_id, x_chunk='spikerates',
                                         time_range=time_range,
                                        **load_kwargs)
        y_preds = []
        y_true = []
        X_flat = X.reshape(-1, np.prod(X.shape[1:]))

        model = LogisticRegression(**log_reg_kwargs)

        print(f"Performing {N_SPLITS}-fold cross-validated logistic regression...")
        for kfold, (trn, tst) in enumerate(splitter.split(X, Y)):
            print(f"Fold {kfold + 1}")

            model.fit(X_flat[trn], Y[trn].ravel())
            y_preds.append(model.predict(X_flat[tst]))
            y_true.append(Y[tst].ravel())

        y_preds = np.hstack(y_preds)
        y_true = np.hstack(y_true)

        pcnt_corr = 100 * np.sum(y_preds == y_true) / len(y_preds)
        print(f"{sess_id} 8-class accuracy: % {pcnt_corr:.2f}")
        accs_out.append(pcnt_corr)
        
    return accs_out

In [ ]:
accs_out_bline = get_accuracies(sess_infos, time_range=(-np.inf, 0))
accs_out_targ = get_accuracies(sess_infos, time_range=(-np.inf, 0.250))
accs_out_prego = get_accuracies(sess_infos, time_range=(-np.inf, 1.45))
accs_out_all = get_accuracies(sess_infos, time_range=(-np.inf, np.inf))

In [ ]:
print(np.vstack((accs_out_bline, accs_out_targ, accs_out_prego, accs_out_all)).T)
print([_['exp_code'] for _ in sess_infos])

## ML Analysis 2 - SVM

In [ ]:
from sklearn import svm


svm_kwargs = {
    'C': 0.1,  # inverse regularization strength
    'kernel': 'linear',  # ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’
#     'degree': 3,  # Only used if kernel is 'poly'
#     'gamma': 'scale',  # kernel coeff, used by 'rbf', 'poly', and 'sigmoid'
    'class_weight': 'balanced',
}

def get_accuracies_svm(sess_infos, time_range=(-np.inf, np.inf),
                       svm_kwargs={'C': 0.1, 'kernel': 'linear',
                                   'class_weight': 'balanced'}):
    
    splitter = StratifiedKFold(n_splits=N_SPLITS, shuffle=True)

    accs_out = []
    for sess_ix, sess_info in enumerate(sess_infos):
        sess_id = sess_info['exp_code']
        print(f"\nProcessing session {sess_id}")
        X, Y, ax_info = load_macaque_pfc(data_path, sess_id, x_chunk='spikerates',
                                         time_range=time_range,
                                        **load_kwargs)
        y_preds = []
        y_true = []
        X_flat = X.reshape(-1, np.prod(X.shape[1:]))

        model = svm.SVC(**svm_kwargs)

        print(f"Performing {N_SPLITS}-fold cross-validated SVM...")
        for kfold, (trn, tst) in enumerate(splitter.split(X, Y)):
            print(f"Fold {kfold + 1}")

            model.fit(X_flat[trn], Y[trn].ravel())
            y_preds.append(model.predict(X_flat[tst]))
            y_true.append(Y[tst].ravel())

        y_preds = np.hstack(y_preds)
        y_true = np.hstack(y_true)

        pcnt_corr = 100 * np.sum(y_preds == y_true) / len(y_preds)
        print(f"{sess_id} 8-class accuracy: % {pcnt_corr:.2f}")
        accs_out.append(pcnt_corr)
        
    return accs_out

In [ ]:
accs_out_bline = get_accuracies(sess_infos, time_range=(-np.inf, 0))
accs_out_targ = get_accuracies(sess_infos, time_range=(-np.inf, 0.250))
accs_out_prego = get_accuracies(sess_infos, time_range=(-np.inf, 1.45))
accs_out_all = get_accuracies(sess_infos, time_range=(-np.inf, np.inf))

In [ ]:
print(np.vstack((accs_out_bline, accs_out_targ, accs_out_prego, accs_out_all)).T)
print([_['exp_code'] for _ in sess_infos])